In [1]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl

# plt.style.use('dark_background')
from mog_fit import data
from collections import OrderedDict
from sklearn import preprocessing

import torch
from torch.autograd import Variable
import torchvision.models as models

import warnings
warnings.filterwarnings("ignore")


%load_ext autoreload
%autoreload 2

# load in imagenet dset

In [14]:
import torchvision.datasets as dset
import torchvision.transforms as transforms

# Data loading code
data_dir = '/scratch/users/vision/data/cv/imagenet_full'
traindir = os.path.join(data_dir, 'train')
valdir = os.path.join(data_dir, 'val')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

batch_size = 10
num_workers = 1
'''
train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=False,
    num_workers=num_workers, pin_memory=True)
'''
val_loader = torch.utils.data.DataLoader(
    dset.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize])),
    batch_size=batch_size, shuffle=False, 
    num_workers=num_workers, pin_memory=True)


KeyboardInterrupt



# load in some models

In [22]:

def get_model(s):
    if s == 'densenet':
        model = models.densenet161(pretrained=True)
    elif s == 'alexnet':
        model = models.alexnet(pretrained=True)
    elif s == 'resnet18':
        model = models.resnet18(pretrained=True)
    elif s == 'vgg16':
        model = models.vgg16(pretrained=True)
    elif s == 'inception_v3':
        model = models.inception_v3(pretrained=True)
    return model.cuda()

# iterate

In [27]:
def linear_hook(module, act_in, act_out):
#     print(module.__class__.__name__)
#     max_corrs[module.name] = 
    print(module.weight)
#     print(module.name)
#     outputs.append(output)
    
def calc_max_corr(val_loader, outputs):
    for x in val_loader:
        ims = x[0].cuda()
        break
    _ = model(Variable(ims))
    

model = get_model('alexnet')    


# list(model.modules())

# pick some layers
lay1 = model.classifier[1] # fc1
lay1.name = 'fc1'
# model.classifier[4] # fc2
# model.classifier[6] # fc3

max_corrs = {}
# loop over hooks / Ws here
model.classifier[1].register_forward_hook(linear_hook)
calc_max_corr(val_loader, outputs)

Parameter containing:
 4.0023e-03  6.0941e-03  1.3376e-02  ...  -3.1656e-04 -2.1764e-03 -9.2843e-03
 1.3021e-03  2.0498e-03  2.7710e-03  ...   5.2761e-03  2.0754e-04 -1.4514e-02
-1.8520e-02 -1.4540e-02 -1.9814e-02  ...  -3.3310e-04 -1.2449e-03  1.9117e-02
                ...                   ⋱                   ...                
-6.3274e-03  2.6374e-03 -8.3830e-03  ...  -7.9338e-03  5.5023e-03  6.6757e-03
-1.1492e-02 -5.5136e-03 -9.2469e-03  ...  -3.8866e-03 -4.8030e-03 -9.8874e-03
 2.6484e-03 -1.4830e-02 -1.6435e-03  ...   5.0250e-03  5.1868e-03 -4.4632e-03
[torch.cuda.FloatTensor of size 4096x9216 (GPU 0)]

